Input: Comerturas KML de la Capa "Uso"
Output: Dataframe

In [4]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd

# Lista con las rutas de todos los archivos KML
kml_files = ['usolaspalmas.kml','usolospinos.kml']  

# Lista para almacenar las características de todos los terrenos
terrenos = []

# Función para procesar un archivo KML
def procesar_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Espacio de nombres KML
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Iterar sobre cada <Placemark>
    placemarks = root.findall('.//kml:Placemark', ns)
    for placemark in placemarks:
        # Extraer el nombre y descripción
        nombre = placemark.find('kml:name', ns).text
        descripcion_html = placemark.find('kml:description', ns).text

        # Procesar la descripción para extraer la segunda tabla
        soup = BeautifulSoup(descripcion_html, 'html.parser')
        tables = soup.find_all('table')
        second_table = tables[1] if len(tables) > 1 else None

        # Almacenar los pares clave-valor de la segunda tabla
        info_terreno = {'nombre': nombre}
        if second_table:
            rows = second_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:  # Esperamos pares clave-valor
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    info_terreno[key] = value

        # Extraer las coordenadas (múltiples puntos delimitan el terreno)
        coordenadas_texto = placemark.find('.//kml:coordinates', ns).text.strip()
        coordenadas = [tuple(map(float, coord.split(','))) for coord in coordenadas_texto.split()]
        info_terreno['coordenadas'] = coordenadas

        terrenos.append(info_terreno)

# Procesar todos los archivos KML
for kml_file in kml_files:
    procesar_kml(kml_file)

# Convertir toda la información a un DataFrame
df_terrenos = pd.DataFrame(terrenos)

# Exportar el DataFrame a Excel
df_terrenos.to_excel('terrenos.xlsx', index=False)

# Exportar el DataFrame a CSV
df_terrenos.to_csv('terrenos.csv', index=False)

print("Datos exportados exitosamente a Excel y CSV.")


Datos exportados exitosamente a Excel y CSV.


In [5]:
df_terrenos.describe()

,nombre,FID,ID_UNICO,ID_PREDIO,NOMBRE,ID_USO,ID_SUBUSO,ID_ESTRUC,ID_TIFO,ID_STIF,...,ID_MANEJO,ID_RALEO,ID_PODA,ID_SANID,ID_ROTAC,USO_COT,USO_CEFOR,OBS,SUP_HA,coordenadas
count,580,580,580,580,580,580,580,580,580,580,...,580,580,580,580,580,580,580,580,580,580
unique,2,318,318,2,2,5,6,5,3,5,...,4,6,5,4,4,15,10,19,248,580
top,01,0,1,01,Los Pinos,04,01,01,,,...,,,,,,Plantacion Adulta,Plantacion,,0.42,"[(-73.14953102003405, -39.74768325608614, 0.0)..."
freq,318,2,2,318,318,427,292,220,463,463,...,378,366,366,366,368,219,258,549,12,1


In [6]:
# prompt: nombres unicos de la columna USO_CEFOR

usos_cefor_unicos = df_terrenos['USO_CEFOR'].unique()
print(usos_cefor_unicos)


['Faja Camino' 'Bosques Mixtos' 'Plantacion' 'Bosque Nativo' 'Matorral'
 'Bosque Exoticas Asilvestradas' 'Reforestacion' 'Pradera'
 'Terrenos Humedos' 'Cuerpos de Agua']
